In [5]:
import pandas as pd
import os
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import shuffle
from sklearn.model_selection import LeaveOneOut
import itertools

file_name = 'Dados_Forchheimer.xlsx'
file_path = os.path.join(os.getcwd(), file_name)

ANNDF = pd.read_excel(file_path, sheet_name='Train')

X = ANNDF.iloc[:, 0:2 ].values
Y = ANNDF.iloc[:, 2].values.reshape(-1, 1)

X, Y = shuffle(X, Y)

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size=0.75)  # 25% for testing and shuffle

scalerx = StandardScaler()
scalery = StandardScaler()

Xtrain = scalerx.fit_transform(xtrain)
Ytrain = scalery.fit_transform(ytrain)

loocv = LeaveOneOut()
loocv.get_n_splits(X)

mlp = MLPRegressor(max_iter=5000)

nneurons = [32, 64, 128, 256]
parameters = {
    'hidden_layer_sizes': [],
    'activation': ['relu', 'logistic'],
    'solver': ['adam']
}

# Gerar configurações de 1 a 4 camadas
for n_layers in range(2, 6):  # 1, 2, 3 e 4 camadas
    for hlayers in itertools.product(nneurons, repeat=n_layers):
        parameters['hidden_layer_sizes'].append(hlayers)

ypred_or = []
ytest_val = []
bestparam = []
bestac    = []

for i, (train_index, test_index) in enumerate(loocv.split(X)):
    print(i+1)
    xtrain = X[train_index,:]
    xtest  = X[test_index,:]

    ytrain = Y[train_index,:]
    ytest  = Y[test_index,:]

    scalerx = StandardScaler()
    scalery = StandardScaler()

    Xtrain = scalerx.fit_transform(xtrain)
    Ytrain = scalery.fit_transform(ytrain)

    kf = KFold(n_splits=5, shuffle=True) #max 5...

    gsearchcv = GridSearchCV(estimator=mlp, param_grid=parameters, cv=kf, verbose=0)
    gsearchcv.fit(Xtrain, Ytrain.ravel())

    bestac.append(gsearchcv.best_score_ * 100)  # Convert to percentage
    bestparam.append(gsearchcv.best_params_)

    print('Best Accuracy: {:.2f} %'.format(bestac[i]))
    print('Best Parameters:', bestparam[i])

    Xtest_scaled = scalerx.transform(xtest)
    ypred = gsearchcv.predict(Xtest_scaled).reshape(-1, 1)
    ypred_or.append(scalery.inverse_transform(ypred))
    ytest_val.append(ytest)

import joblib

# Após o treinamento do modelo
joblib.dump(gsearchcv.best_estimator_, 'mlp_model_forch_loocv.pkl')
joblib.dump(scalerx, 'scalerx_mlp_model_forch.pkl')
joblib.dump(scalery, 'scalery_mlp_model_forch.pkl')

1
Best Accuracy: 99.90 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (128, 256, 128, 256, 64), 'solver': 'adam'}
2
Best Accuracy: 99.82 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (32, 256, 64, 64, 256), 'solver': 'adam'}
3
Best Accuracy: 99.85 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (64, 256, 64, 64, 128), 'solver': 'adam'}
4
Best Accuracy: 99.62 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (128, 128, 256, 256, 256), 'solver': 'adam'}
5
Best Accuracy: 99.94 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (256, 128, 256, 64, 32), 'solver': 'adam'}
6
Best Accuracy: 99.92 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (128, 64, 256, 256, 256), 'solver': 'adam'}
7
Best Accuracy: 99.94 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (256, 128, 256, 128, 128), 'solver': 'adam'}
8
Best Accuracy: 99.91 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes':

['scalery_mlp_model_forch.pkl']

In [6]:
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_percentage_error

ypred_or = [items for subList in ypred_or for items in subList]
ytest_val = [items for subList in ytest_val for items in subList]

r2 = r2_score(ytest_val, ypred_or)
rmse = root_mean_squared_error(ytest_val, ypred_or)
mape = mean_absolute_percentage_error(ytest_val, ypred_or)

In [13]:
import joblib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

mlp_model = joblib.load('mlp_model_forch_loocv.pkl')
scalerx = joblib.load('scalerx_mlp_model_forch.pkl')
scalery = joblib.load('scalery_mlp_model_forch.pkl')

import numpy as np

novo_valor = np.array([[8497, 0.4476]])  # Note que o formato deve ser (1, n_features)

novo_valor_scaled = scalerx.transform(novo_valor)

pred_scaled = mlp_model.predict(novo_valor_scaled)

pred = scalery.inverse_transform(pred_scaled.reshape(-1, 1))

print("Predição:", pred)

Predição: [[-24841.02460753]]
